# Svm

In [10]:
import numpy as np
import sklearn.utils.extmath as sm
from numpy.linalg import inv
from numpy.linalg import eig
from scipy.sparse.linalg import eigs
from numpy import dot, diag
from scipy.linalg import sqrtm
from scipy.spatial.distance import euclidean
import pandas as pd
import random, math
np.random.seed(42)
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pickle


# Cluster kernel

## K Matrix Paper

In [2]:
def fill_diag(M, a):
    """
    M: square matrix
    a: array of length number of rows
    ----
    fill the diagonal of M with values of array a
    """
    s = M.shape
    D = np.zeros(s)
    for i in range(s[0]):
        D[i,i] = a[i]
    return D

def rbf_kernel(X, sigma=1):
    K = np.zeros((len(X), len(X)))
    for a in range(len(X)):
        for b in range(len(X)):
            K[a, b] = rbf_function(X[a], X[b],sigma)
    return K
            
def rbf_function(x, y, sigma=1):
    exponent = - (euclidean(x, y) ** 2) / (2 * (sigma ** 2))
    return np.exp(exponent)


def diagonal_row_sum_matrix(M):
    rows_sum = M.sum(axis = 1)
    return fill_diag(M,rows_sum)

def computeL(D,K):
    Dinv = inv(D)
    return sqrtm(Dinv).dot(K).dot(sqrtm(Dinv))

def pick_eigs(eigen_vals, eigen_vect, k):
    eig_vals = list(eigen_vals)
    new_eigs = []
    new_eigen_vector = []
    last_max_value = 0
    for i in range(0, k):
        argmax = eig_vals.index(max(eig_vals))
        new_eigen_vector.append(eigen_vect[argmax])
        new_eig = eig_vals.pop(argmax)
        new_eigs.append(new_eig)
        last_max_value = new_eig
        
    argmax = eig_vals.index(max(eig_vals))
    new_eig = eig_vals[argmax]
        
    while(new_eig == last_max_value):
        argmax = eig_vals.index(max(eig_vals))
        new_eigen_vector.append(eigen_vect[argmax])
        new_eig = eig_vals.pop(argmax)
        new_eigs.append(new_eig)
        
    return np.array(new_eigs), np.array(new_eigen_vector)      
    

def build_K(lambdaCut, transfer, X, n_clusters, sigma=0.5):
    
    #Step 1 - K matrix
    K = rbf_kernel(X, sigma)
    D = diagonal_row_sum_matrix(K)
    
    #Step 2 - L matrix
    L = computeL(D, K)
    eigen_vals, U = eig(L)
    
    Q = diag(eigen_vals)
    
    #Step 3 - Transfer Function
    #choosing lambdacut
    newEigen = transfer(eigen_vals, lambdaCut)
    newEigen = diag(newEigen)
    
    #Step 4 - New Kernel matrix
    newL = (U).dot(newEigen).dot(U.T)
    newD = inv(diag(diag(L)))
    newK = sqrtm(newD).dot(newL).dot(sqrtm(newD))
    return newK

def build_K_inverse(lambdaCut, transfer, X, n_clusters, sigma=0.5):
    
    #Step 1 - K matrix
    K = rbf_kernel(X, sigma)
    D = diagonal_row_sum_matrix(K)
    
    #Step 2 - L matrix
    L = computeL(D, K)
    eigen_vals, U = eig(L)
    
    
    Q = diag(eigen_vals)
    
    #Step 3 - Transfer Function
    #choosing lambdacut
    newEigen = transfer(eigen_vals, lambdaCut)
    newEigen = diag(newEigen)
    
    #Step 4 - New Kernel matrix
    newL = (U).dot(newEigen).dot(np.linalg.inv(U))
    newD = inv(diag(diag(L)))
    newK = sqrtm(newD).dot(newL).dot(sqrtm(newD))
    return newK
    

#TRANSFER FUNCTION
def linear(vals, lambdaCut):
    return vals

def step(vals,lambdaCut):
    return [ 1 if x >= lambdaCut else 0 for x in vals ]

def linear_step(vals, lambdaCut):
    return [ x if x >= lambdaCut else 0 for x in vals ]

def polynomial(vals, exponent):
    return [ np.power(x, exponent) for x in vals ]

def polystep(vals, lambdaCut):
    return [ np.power(x, 2) if x > lambdaCut else np.sqrt(x) for x in vals ]
    
    

In [3]:
#dummy example
nb_samples = 6 #nb of samples
dim_sample = 4
X = np.random.rand(nb_samples,dim_sample)
# print(X)
lambdaCut = 1
K = build_K(lambdaCut, linear, X, 2)
# print(K)
K = build_K_inverse(lambdaCut, linear, X, 2)
# print(K)

In [4]:
K = build_K(lambdaCut, linear, X, 2, sigma=39)
print(K)

[[1.         0.99960394 0.99975212 0.99959574 0.99987807 0.99968387]
 [0.99960394 1.         0.9998307  0.99969031 0.99979348 0.99983149]
 [0.99975212 0.9998307  1.         0.99968977 0.99975297 0.99974973]
 [0.99959574 0.99969031 0.99968977 1.         0.99985182 0.99996726]
 [0.99987807 0.99979348 0.99975297 0.99985182 1.         0.99991182]
 [0.99968387 0.99983149 0.99974973 0.99996726 0.99991182 1.        ]]


## Linear Classifier  - SVM functions


In [5]:
### Kernels
#Defining a linear Kernel
def kernel_1(li1, li2):
    #assuming that li1 and li2 are numpy arrays
    scalar_product = np.sum(li1 * li2)
    return scalar_product

#defining a polynomial kernel with degree p
def kernel_2(li1, li2, p):
    scalar_product=np.sum(li1 * li2)
    value = math.pow((scalar_product + 1), p)
    return value

#distance function for Gaussian Kernel
def dist(l1,l2):
    li3 = l1 - l2
    li3 = li3 * li3
    dist_sq = np.sum(li3)
    return dist_sq


#defining a gaussian kernel with parameter gamma
def kernel_3(li1, li2, gamma):
    Distance_sq = dist(li1, li2)
    temp = gamma * Distance_sq
    value = math.exp( - temp)
    return value

def choose_kernel(num, li1, li2, deg = 0, gamma = 0):
    if(num == 1):
        return kernel_1(li1, li2)
    elif(num == 2):
        return kernel_2(li1, li2, deg)
    else:
        return kernel_3(li1, li2, gamma)
    
### SVM inherent functions
# Global matrix - needs to be as a global variable
def pre_cal(features_list, target_list, kernel, deg, gamma):
    
    P_matrix = []
    
    for i in range(len(target_list)):
        tmp_li = []
        for j in range(len(target_list)):
            tmp_li.append(target_list[i] * target_list[j] * choose_kernel(kernel, features_list[i], features_list[j], deg, gamma))
        tmp_li = np.array(tmp_li)
        P_matrix.append(tmp_li) #p matrix has been created

    return P_matrix

# Constraint functions
def zerofun(alpha_list):
    return np.dot(alpha_list, target_list) #this is the value which should be constrained to zero

# Objective Function
def objective(alpha_list):
    
    sum1=0
    sum2=0    
    for i in range(len(alpha_list)):
        sum1 += alpha_list[i]
        sum2 += alpha_list[i] * np.dot(alpha_list,P_mat[i]) #here alpha list is assumed to be numpy array

    sum2 = sum2 / 2

    return (sum2 - sum1)

# Indicator Function
def indicator(new_data, features_list, target_list, kernel, alpha_list, b, deg, gamma):
    somme = 0
    #print(alpha_list)
    for i in range(len(alpha_list)):
        if(alpha_list[i] != 0):
            somme += alpha_list[i] * target_list[i] * choose_kernel(kernel, new_data, features_list[i], deg, gamma)
        else:
            continue

    ind = (somme - b)

    return ind


#Return B
def get_b(alphas, non_zeros_indices, inputs, targets, kernel, p_or_sigma = 0):
    b=[]
    bb = 0
    
    for i in non_zeros_indices:
        bb = 0
        for j in range(0, len(inputs)):
            bb += alphas[j] * targets[j] * kernel(inputs[i], inputs[j], p_or_sigma)
        bb -= targets[i]
        b.append(bb)  
    
    return np.mean(b)



# Plot
def plot(ClassA, ClassB, filtered_alphas, b, kernel, slack, deg, gamma):

    plt.plot([p[0] for p in ClassA],[p[1] for p in ClassA],'b.')
    plt.plot([p[0] for p in ClassB],[p[1] for p in ClassB],'r.')
    plt.axis('equal')

    xgrid = np.linspace(-5,5)  #by default it is 50 in numpy
    ygrid = np.linspace(-4,4)  


    grid = np.array([[indicator(np.array([x,y]), features_list, target_list, kernel, filtered_alphas,b, deg, gamma) for x in xgrid]for y in ygrid])
    #print(grid)
    plt.contour(xgrid,ygrid,grid,(-1,0,1),colors=('red','black','blue'),linewidths=(1,3,1))
    plt.xlabel("Feature1")
    plt.ylabel("Feature2")
    plt.title("Decision Boundary with Margins with C value : "+str(slack))
    #plt.savefig("part4_rbf_1_C_"+str(slack)+".png")
    plt.show()

In [6]:
def train(features_list, target_list, kernel, slack, deg = 0, gamma = 0):
    
    N = len(features_list)
    non_zero_indices = []
    
    # Minimising Function
    ret = minimize(objective,np.zeros(N),bounds=[(0,slack) for b in range(features_list.shape[0])],constraints={'type':'eq','fun':zerofun})
    alphas = ret['x']
    
    # Retreive Non zero alphas
    filtered_alphas = []
    for i in range(len(alphas)):
        #if the alpha is less 10-5 then i will consider it as 0 === TRESHOLD
        if(alphas[i]<math.pow(10,-5)):
            filtered_alphas.append(0)
        else:
            if alphas[i] <= slack:
                non_zero_indices.append(i)
            support_vec=features_list[i]
            support_target=target_list[i]
            filtered_alphas.append(alphas[i])
            
    # Return b's
    b = get_b(filtered_alphas, non_zero_indices, features_list, target_list, kernel, deg)
    
    
    return filtered_alphas, b, non_zero_indices 

def test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg = 0, gamma = 0 ):
    
    predictions = []
    for instance in range(0, len(test_features)):
        prediction = indicator(test_features[instance], train_features, train_targets, kernel, alpha_list, b, deg, gamma)
        predictions.append(prediction)
    
    temp = []
    for prediction in predictions:
        if prediction > 0:
            temp.append(1)
        else:
            temp.append(-1)
    predictions = temp
    
    true = np.array(test_targets)
    preds = np.array(predictions)
    error = np.sum(test_targets != predictions)
    
    return predictions, error

## Logistic Regression

In [7]:
def logistic_function(output):
    return 1 / ( 1 + np.exp( - output ) )

def update_coefficient(curr_coefficient, prediction, feature, target, learning_rate):
    new_coeff = curr_coefficient + ( learning_rate * ( target - prediction ) * prediction * ( 1 - prediction ) * feature )
    return new_coeff

def update_coefficients(coefficients, b0, prediction, instance, target, learning_rate):
    new_coeffs = []
    for coefficient in range(len(coefficients)):
        new_coeff = update_coefficient(coefficients[coefficient], prediction, instance[coefficient], target, learning_rate)
        new_coeffs.append(new_coeff)
    newB = update_coefficient(b0, prediction, 1, target, learning_rate)
    return new_coeffs, newB

def prediction(x, coefficients, b0):
    output = np.sum( np.array(coefficients) * np.array(x) ) + b0
    prediction = logistic_function(output)
    
    return prediction
    


def epoch_logistic_regression(inputs, targets, coefficients, b0):
    predictions = []
    for instance in range(len(inputs)):
        output = np.sum( np.array(coefficients) * np.array(inputs[instance]) ) + b0
        prediction = logistic_function(output)
        coefficients, b0 = update_coefficients(coefficients, b0, prediction, inputs[instance], targets[instance], 0.3)
        predictions.append(prediction)
        
    predictions = [0 if x < 0.5 else 1 for x in predictions]
    
    accuracy = (np.sum(np.array(predictions) == np.array(targets))) / len(targets)

    return coefficients, b0, accuracy

def logistic_regression(inputs, targets):
    coefficients = np.zeros(len(inputs[0]))
    b0 = 0
    
    old_accuracy = 0
    curr_accuracy = 50
    diff = 50
    
    i = 0
    
    while(diff > 0):
        coefficients, b0, curr_accuracy = epoch_logistic_regression(inputs, targets, coefficients, b0)
        diff = curr_accuracy - old_accuracy
        diff = np.abs(diff)
        
        old_accuracy = curr_accuracy
        i+=1
        
    return coefficients, b0, old_accuracy

def test_logistic(inputs, targets, coefficients, b0):
    
    predictions = []
    
    for i in range(0, len(inputs)):
        pred = prediction(inputs[i], coefficients, b0)
        predictions.append(pred)
        
    predictions = [0 if x < 0.5 else 1 for x in predictions]
    error = (np.sum(np.array(predictions) != np.array(targets))) 
    return predictions, error
        
        
    
    
        
        

# Digit dataset

In [11]:
import pickle
from numpy.core import multiarray
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [9]:
with open("../Dataset/digitDataset.pickle", "rb") as fp:
    digits = pickle.load(fp,encoding='bytes')
    
# SVM compatible outputs
for i in range(0, len(digits)):
    if digits[i][1] == 0:
        digits[i][1] = -1.
    else:
        digits[i][1] = 1.
        
# Permutations to shuffle the dataset
permute=list(range(len(digits)))
random.shuffle(permute)
digits = np.array(digits)
digits = digits[permute, :]

# Separate Targets from features
Xs = []
labels = []
for i in range(0, len(digits)):
    Xs.append(digits[i][0])
    labels.append(digits[i][1])

In [10]:
scaler = StandardScaler()
new_Xs = scaler.fit_transform(Xs)
Xs = new_Xs

## Independent main functions

n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
size_test = 2000 - n_labeled
n_clusters = 2

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

# Building k_matrix
lambdaCut = n_labeled + 10;
k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 0.55) 

# Subset the data (50-fold cross validation)
X_k = {}
labels_k = {}
j = -1
for i in range(0, len(digits)):

    if i%40 == 0:
        if i != 0:
            X_k[j] = np.array(X_k[j])
            labels_k[j] = np.array(labels_k[j])
        j += 1
        X_k[j] = []
        labels_k[j] = []

    X_k[j].append(k_matrix[i])
    labels_k[j].append(labels[i])
    
X_k[j] = np.array(X_k[j])
labels_k[j] = np.array(labels_k[j])
    
total_error = {}

kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 5
for fold in X_k:
    total_error[fold] = 0

    # pre_cal(features_list, target_list, kernel, deg, gamma)
    target_list = labels_k[fold]

    P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)

    # train(features_list, target_list, kernel, slack, degree, gamma)
    alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)

    for test_fold in X_k:
        if test_fold != fold:
            # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
            targets_list = labels_k[test_fold]
            test_predict, error = test_score_svm(X_k[test_fold], targets_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
            total_error[fold] += error / size_test
    print("fold number ", fold, "has an error of ", total_error[fold])

## Function to experiment 

### Different polystep cutoffs

In [13]:

################################ RAN
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 5

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

old_predict = None
polystep_error = {}
for lambdaCut in range(6, 55):
    current_predict = []
    total_error = {}
    print("Trying with lambdaCut : ", lambdaCut)
    # Building k_matrix
    k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
    print("     Matrix built!")

    # Subset the data (50-fold cross validation)
    X_k = {}
    labels_k = {}
    j = -1
    for i in range(0, len(digits)):

        if i%40 == 0:
            if i != 0:
                X_k[j] = np.array(X_k[j])
                labels_k[j] = np.array(labels_k[j])
            j += 1
            X_k[j] = []
            labels_k[j] = []

        X_k[j].append(k_matrix[i])
        labels_k[j].append(labels[i])

    X_k[j] = np.array(X_k[j])
    labels_k[j] = np.array(labels_k[j])
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
                test_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], test_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
                current_predict += list(test_predict)
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error for ", lambdaCut, " :",  np.mean(curr_error))
    if current_predict == old_predict:
        print("Same predictions")
    old_predict = current_predict
    polystep_error[lambdaCut] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_errors.pickle", "wb") as fp:
        pickle.dump(polystep_error, fp)
        fp.close()

    

Trying with lambdaCut :  6
     Matrix built!
Mean Test Error for  6  : 0.4473775510204081
Trying with lambdaCut :  7
     Matrix built!
Mean Test Error for  7  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  8
     Matrix built!
Mean Test Error for  8  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  9
     Matrix built!
Mean Test Error for  9  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  10
     Matrix built!
Mean Test Error for  10  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  11
     Matrix built!
Mean Test Error for  11  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  12
     Matrix built!
Mean Test Error for  12  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  13
     Matrix built!
Mean Test Error for  13  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  14
     Matrix built!
Mean Test Error for  14  : 0.4473775510204081
Same predictions
Trying with lambdaCut :  15
     Ma

In [11]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 5

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

old_predict = None
polystep_error = {}
for lambdaCut in np.arange(-10, 10, 2):
    current_predict = []
    total_error = {}
    print("Trying with lambdaCut : ", lambdaCut)
    # Building k_matrix
    k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
    print("     Matrix built!")

    # Subset the data (50-fold cross validation)
    X_k = {}
    labels_k = {}
    j = -1
    for i in range(0, len(digits)):

        if i%40 == 0:
            if i != 0:
                X_k[j] = np.array(X_k[j])
                labels_k[j] = np.array(labels_k[j])
            j += 1
            X_k[j] = []
            labels_k[j] = []

        X_k[j].append(k_matrix[i])
        labels_k[j].append(labels[i])

    X_k[j] = np.array(X_k[j])
    labels_k[j] = np.array(labels_k[j])
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
                test_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], test_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
                current_predict += list(test_predict)
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error for ", lambdaCut, " :",  np.mean(curr_error))
    if current_predict == old_predict:
        print("Same predictions")
    old_predict = current_predict
    polystep_error[lambdaCut] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_errorsBis.pickle", "wb") as fp:
        pickle.dump(polystep_error, fp)
        fp.close()

    

Trying with lambdaCut :  -10
     Matrix built!
Mean Test Error for  -10  : 0.27403061224489794
Trying with lambdaCut :  -8
     Matrix built!
Mean Test Error for  -8  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  -6
     Matrix built!
Mean Test Error for  -6  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  -4
     Matrix built!
Mean Test Error for  -4  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  -2
     Matrix built!
Mean Test Error for  -2  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  0
     Matrix built!
Mean Test Error for  0  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  2
     Matrix built!
Mean Test Error for  2  : 0.4531734693877551
Trying with lambdaCut :  4
     Matrix built!
Mean Test Error for  4  : 0.4531734693877551
Same predictions
Trying with lambdaCut :  6
     Matrix built!
Mean Test Error for  6  : 0.4531734693877551
Same predictions
Trying with lambdaCut :  8
     Matrix built

In [13]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 5

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

old_predict = None
polystep_error = {}
for lambdaCut in np.arange(0, 2, 0.25):
    current_predict = []
    total_error = {}
    print("Trying with lambdaCut : ", lambdaCut)
    # Building k_matrix
    k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
    print("     Matrix built!")

    # Subset the data (50-fold cross validation)
    X_k = {}
    labels_k = {}
    j = -1
    for i in range(0, len(digits)):

        if i%40 == 0:
            if i != 0:
                X_k[j] = np.array(X_k[j])
                labels_k[j] = np.array(labels_k[j])
            j += 1
            X_k[j] = []
            labels_k[j] = []

        X_k[j].append(k_matrix[i])
        labels_k[j].append(labels[i])

    X_k[j] = np.array(X_k[j])
    labels_k[j] = np.array(labels_k[j])
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
                test_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], test_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
                current_predict += list(test_predict)
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error for ", lambdaCut, " :",  np.mean(curr_error))
    if current_predict == old_predict:
        print("Same predictions")
    old_predict = current_predict
    polystep_error[lambdaCut] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_errorsTris.pickle", "wb") as fp:
        pickle.dump(polystep_error, fp)
        fp.close()

    

Trying with lambdaCut :  0.0
     Matrix built!
Mean Test Error for  0.0  : 0.27403061224489794
Trying with lambdaCut :  0.25
     Matrix built!
Mean Test Error for  0.25  : 0.43957142857142856
Trying with lambdaCut :  0.5
     Matrix built!
Mean Test Error for  0.5  : 0.45830612244897956
Trying with lambdaCut :  0.75
     Matrix built!
Mean Test Error for  0.75  : 0.4659693877551021
Trying with lambdaCut :  1.0
     Matrix built!
Mean Test Error for  1.0  : 0.45297959183673475
Trying with lambdaCut :  1.25
     Matrix built!
Mean Test Error for  1.25  : 0.4531734693877551
Trying with lambdaCut :  1.5
     Matrix built!
Mean Test Error for  1.5  : 0.4531734693877551
Same predictions
Trying with lambdaCut :  1.75
     Matrix built!
Mean Test Error for  1.75  : 0.4531734693877551
Same predictions


In [14]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 5

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

old_predict = None
polystep_error = {}
for lambdaCut in np.arange(-0.1, 1.5, 0.05):
    current_predict = []
    total_error = {}
    print("Trying with lambdaCut : ", lambdaCut)
    # Building k_matrix
    k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
    print("     Matrix built!")

    # Subset the data (50-fold cross validation)
    X_k = {}
    labels_k = {}
    j = -1
    for i in range(0, len(digits)):

        if i%40 == 0:
            if i != 0:
                X_k[j] = np.array(X_k[j])
                labels_k[j] = np.array(labels_k[j])
            j += 1
            X_k[j] = []
            labels_k[j] = []

        X_k[j].append(k_matrix[i])
        labels_k[j].append(labels[i])

    X_k[j] = np.array(X_k[j])
    labels_k[j] = np.array(labels_k[j])
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
                test_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], test_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
                current_predict += list(test_predict)
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error for ", lambdaCut, " :",  np.mean(curr_error))
    if current_predict == old_predict:
        print("Same predictions")
    old_predict = current_predict
    polystep_error[lambdaCut] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_errorsTris.pickle", "wb") as fp:
        pickle.dump(polystep_error, fp)
        fp.close()

    

Trying with lambdaCut :  -0.1
     Matrix built!
Mean Test Error for  -0.1  : 0.27403061224489794
Trying with lambdaCut :  -0.05
     Matrix built!
Mean Test Error for  -0.05  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  0.0
     Matrix built!
Mean Test Error for  0.0  : 0.27403061224489794
Same predictions
Trying with lambdaCut :  0.05000000000000002
     Matrix built!
Mean Test Error for  0.05000000000000002  : 0.38584693877551013
Trying with lambdaCut :  0.1
     Matrix built!
Mean Test Error for  0.1  : 0.4150000000000001
Trying with lambdaCut :  0.15
     Matrix built!
Mean Test Error for  0.15  : 0.42482653061224485
Trying with lambdaCut :  0.20000000000000004
     Matrix built!
Mean Test Error for  0.20000000000000004  : 0.43013265306122456
Trying with lambdaCut :  0.25
     Matrix built!
Mean Test Error for  0.25  : 0.43957142857142856
Trying with lambdaCut :  0.30000000000000004
     Matrix built!
Mean Test Error for  0.30000000000000004  : 0.440193877551020

###  Normal Svm

#### Own SVM implementation

In [12]:
# Subset the data (50-fold cross validation)
X_svm = {}
labels_svm = {}
j = -1
for i in range(0, len(digits)):

    if i%40 == 0:
        if i != 0:
            X_svm[j] = np.array(X_svm[j])
            labels_svm[j] = np.array(labels_svm[j])
        j += 1
        X_svm[j] = []
        labels_svm[j] = []

    X_svm[j].append(Xs[i])
    labels_svm[j].append(labels[i])

X_svm[j] = np.array(X_svm[j])
labels_svm[j] = np.array(labels_svm[j])

total_error = {}
for fold in X_svm:
    total_error[fold] = 0

    # pre_cal(features_list, target_list, kernel, deg, gamma)
    target_list = labels_svm[fold]

    P_mat = pre_cal(X_svm[fold], target_list, kernel_svm_no, degree, gamma)

    # train(features_list, target_list, kernel, slack, degree, gamma)
    alpha, b, sv_indices = train(X_svm[fold], target_list, kernel_svm, slack, degree, gamma)

    for test_fold in X_k:
        if test_fold != fold:
            # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
            targets_list = labels_svm[test_fold]
            test_predict, error = test_score_svm(X_svm[test_fold], targets_list, X_svm[fold], labels_svm[fold], kernel_svm_no, alpha, b, degree, gamma )
#             print(test_predict)
            print(test_predict)
            print(targets_list)
            total_error[fold] += error / size_test
#             print(error)
    print("Mean Test Error ", total_error[fold])
    with open("./scaled_nofakelambda_noselectioneigs_errors_svm_errors.pickle", "wb") as fp:
        pickle.dump(total_error, fp)
        fp.close()

[1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[-1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1]
[-1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.
 -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.
  1. -1. -1. -1.]
[-1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1

[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[-1, 1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1]
[-1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.
 -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.
  1. -1. -1. -1.]
[1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1]
[ 1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1. -1.
  1. -1. -1.  1. -1.  1. -1. -1. -1.  1.  1.  1. -1. -1.  1.  1.  1. -1.
  1. -1.  1.  1.]
[1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1]
[ 1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.
 -1.

[1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1]
[-1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.
 -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.
  1. -1. -1. -1.]
[-1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1,

[1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 

[1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1

...

[-1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, -1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1,

[1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, -1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[-1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1

  1. -1.  1.  1.]
[1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[-1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1]
[-1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.
 -1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.
  1. -1. -1. -1.]
[1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1,

[1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1]
[ 1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.
  1. -1.  1.  1.]
[1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1, -1]
[ 1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
 -1. -1.  1.  1.]
[-1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, 1]
[-1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.
  1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1.]
[-1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1,

KeyboardInterrupt: 

In [ ]:
len(Xs[0])

#### Scikit SVM

In [18]:
from sklearn import svm
import pickle
import random
import math

# Load dataset
with open("../Dataset/digitDataset.pickle", "rb") as fp:
	digits = pickle.load(fp,encoding='bytes')
# list of testerror for every run
acclist = []

# compute 100 tuns
for z in range(0,100):
    # tale 8 samples from class 1 and 2 for fitting process
    trainSamples = []
    trainSamples += random.sample(digits[0:1000],8)
    trainSamples += random.sample(digits[1001:],8)
    trainX = []
    trainY = []
    for x in trainSamples:
        trainX.append(x[0])
        trainY.append(x[1])
    clf = svm.SVC(gamma='scale')
    clf.fit(trainX,trainY)

    # calulate testerror
    corrects = 0.0
    count = 0.0
    for x in digits:
        pred = clf.predict(x[0].reshape(1,-1))
        count+=1.0
        if(pred == x[1]):
            corrects += 1.0
    acclist.append((count-corrects)/count)
print("mean: ")
mean = sum(acclist)/100.0
print(mean)
print("standard deviation: ")
der = 0
sq = []
for x in acclist:
    sq.append((x-mean)**2)
sqmean = sum(sq)/100.0
print(math.sqrt(sqmean))


mean: 
0.3609599999999999
standard deviation: 
0.05024553114457045


### Different cutoffs with logistic regression


In [8]:
with open("../Dataset/digitDataset.pickle", "rb") as fp:
    digits = pickle.load(fp,encoding='bytes')
    
# SVM compatible outputs
for i in range(0, len(digits)):
    if digits[i][1] == 0:
        digits[i][1] = 0.
    else:
        digits[i][1] = 1.
        
# Permutations to shuffle the dataset
permute=list(range(len(digits)))
random.shuffle(permute)
digits = np.array(digits)
digits = digits[permute, :]

# Separate Targets from features
Xs = []
labels = []
for i in range(0, len(digits)):
    Xs.append(digits[i][0])
    labels.append(digits[i][1])

In [12]:
scaler = StandardScaler()
new_Xs = scaler.fit_transform(Xs)
Xs = new_Xs

In [16]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
size_test = 2000 - n_labeled
n_clusters = 2

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''

old_predict = None
polystep_error = {}
for lambdaCut in range(0, 2, 0.2):
    current_predict = []
    total_error = {}
    print("Trying with lambdaCut : ", lambdaCut)
    # Building k_matrix
    k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
    print("     Matrix built!")

    # Subset the data (50-fold cross validation)
    X_k = {}
    labels_k = {}
    j = -1
    for i in range(0, len(digits)):

        if i%40 == 0:
            if i != 0:
                X_k[j] = np.array(X_k[j])
                labels_k[j] = np.array(labels_k[j])
            j += 1
            X_k[j] = []
            labels_k[j] = []

        X_k[j].append(k_matrix[i])
        labels_k[j].append(labels[i])

    X_k[j] = np.array(X_k[j])
    labels_k[j] = np.array(labels_k[j])
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        coefficients, b0, accuracy = logistic_regression(X_k[fold], target_list)

        for test_fold in X_k:
            if test_fold != fold:
                # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
                targets_list = labels_k[test_fold]
                predictions, error = test_logistic(X_k[test_fold], targets_list, coefficients, b0)
                current_predict += list(test_predict)
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error for ", lambdaCut, " :",  np.mean(curr_error))
    if current_predict == old_predict:
        print("Same predictions")
    old_predict = current_predict
    polystep_error[lambdaCut] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_polystep_errors_logistic.pickle", "wb") as fp:
        pickle.dump(polystep_error, fp)
        fp.close()

    

    

Trying with lambdaCut :  6
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  7
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  8
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  9
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  10
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  11
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  12
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  13
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  14
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  15
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  16
     Matrix built!
Mean Test Error  0.3589081632653061
Trying with lambdaCut :  17
     Matrix built!
Mean Test Error  0.3589081632653061
Trying w

### Different parameters for the svm

#### Polynomial

In [ ]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
lambdaCut = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
degree = [2,3,4,5,6,7]
gamma = 0
# gamma = [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''



# Building k_matrix
k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
print("     Matrix built!")

# Subset the data (50-fold cross validation)
X_k = {}
labels_k = {}
j = -1
for i in range(0, len(digits)):

    if i%40 == 0:
        if i != 0:
            X_k[j] = np.array(X_k[j])
            labels_k[j] = np.array(labels_k[j])
        j += 1
        X_k[j] = []
        labels_k[j] = []

    X_k[j].append(k_matrix[i])
    labels_k[j].append(labels[i])

X_k[j] = np.array(X_k[j])
labels_k[j] = np.array(labels_k[j])

    
degree_error = {}

for deg in degree:

    total_error = {}
    
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, deg, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, deg, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                targets_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], targets_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, deg, gamma )
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error ", np.mean(curr_error))
    degree_error[deg] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_degree_svm_hpm_errors.pickle", "wb") as fp:
        pickle.dump(degree_error, fp)
        fp.close()

    

In [ ]:
degree_error

#### Gamma

In [20]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
features = Xs
labels = labels
slack = 10
lambdaCut = 10
size_test = 2000 - n_labeled
n_clusters = 2

## Fixed parameters for the svm
kernel_svm_no = 3
kernel_svm = kernel_3
# degree = [2,3,4,5,6,7]
# gammas = 0
deg = 0
gammas = [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 10, 15, 18, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 80, 90, 100]

'''
n_labeled : Number of labeled instances
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
features : original feature data
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm

'''



# Building k_matrix
k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 5) 
print("     Matrix built!")

# Subset the data (50-fold cross validation)
X_k = {}
labels_k = {}
j = -1
for i in range(0, len(digits)):

    if i%40 == 0:
        if i != 0:
            X_k[j] = np.array(X_k[j])
            labels_k[j] = np.array(labels_k[j])
        j += 1
        X_k[j] = []
        labels_k[j] = []

    X_k[j].append(k_matrix[i])
    labels_k[j].append(labels[i])

X_k[j] = np.array(X_k[j])
labels_k[j] = np.array(labels_k[j])

    
gamma_error = {}

for gamma in gammas:

    total_error = {}
    
    
    for fold in X_k:
        total_error[fold] = 0

        # pre_cal(features_list, target_list, kernel, deg, gamma)
        target_list = labels_k[fold]

        P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, deg, gamma)

        # train(features_list, target_list, kernel, slack, degree, gamma)
        alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, deg, gamma)

        for test_fold in X_k:
            if test_fold != fold:
                targets_list = labels_k[test_fold]
                test_predict, error = test_score_svm(X_k[test_fold], targets_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, deg, gamma )
                total_error[fold] += error / size_test
    curr_error = list(total_error.values())
    print("Mean Test Error ", np.mean(curr_error))
    gamma_error[gamma] = curr_error
    with open("./scaled_nofakelambda_noselectioneigs_gamma_svm_hpm_errors.pickle", "wb") as fp:
        pickle.dump(gamma_error, fp)
        fp.close()

    

     Matrix built!
Mean Test Error  0.4819387755102041
Mean Test Error  0.4845204081632653
Mean Test Error  0.49147959183673473
Mean Test Error  0.49594897959183676
Mean Test Error  0.4957448979591837
Mean Test Error  0.49687755102040826
Mean Test Error  0.4991836734693878
Mean Test Error  0.49658163265306127
Mean Test Error  0.49728571428571433
Mean Test Error  0.49738775510204086
Mean Test Error  0.49724489795918375
Mean Test Error  0.4960714285714286
Mean Test Error  0.49605102040816335
Mean Test Error  0.4956836734693878
Mean Test Error  0.4979591836734694
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
Mean Test Error  0.5012857142857143
M

### Different distributions in the training

In this part, we extend the research by trying out several imbalance levels :
0, 10, 30, 50, 70, 90, 100
with the same methodology (40 training instances and the rest as a test)

In [25]:
# Build K matrix
kernel_k = polystep
slack = 10
lambdaCut = 50
n_clusters = 2
k_matrix = build_K(lambdaCut, kernel_k, Xs, n_clusters, sigma = 5) 

# Separate the classes
Xs0 = []
Xs1 = []
for instance in range(0, len(Xs)):
    if labels[instance] == -1:
        Xs0.append(k_matrix[instance])
    elif labels[instance] == 1:
        Xs1.append(k_matrix[instance])

labels0 = - 1. * np.ones(len(Xs0))
labels1 = np.ones(len(Xs1))

In [ ]:
import random
# Select the instances
def select_percentage(Xs0, Xs1, percentage_1, seed):
    random.seed(seed)
    x0 = Xs0.copy()
    x1 = Xs1.copy()
    
    length = len(Xs0) + len(Xs1)
    n_instances1 = int((40 / 100) * percentage_1)
    n_instances0 = 40 - n_instances1
    
    trainingX = []
    trainingY = []    
    
    for i in range((n_instances1)):
        random_index = random.randint(0, len(x1) - 1)
        instance = x1.pop(random_index)
        trainingX.append(instance)
        trainingY.append(1)
        
    for i in range((n_instances0)):
        random_index = random.randint(0, len(x0) - 1)
        instance = x0.pop(random_index)
        trainingX.append(instance)
        trainingY.append(-1)
        
    y_test0 = list(-1 * np.ones(len(x0)))
    y_test1 = list(np.ones(len(x1)))
    
    test_x = x0 + x1
    test_y = y_test0 + y_test1
    return np.array(trainingX), np.array(trainingY), np.array(test_x), np.array(test_y)        
    

In [ ]:
## Fixed parameters for the cluster kernel algorithm
n_labeled = 40
kernel_k = polystep
slack = 10
lambdaCut = 50
size_test = 2000 - n_labeled

## Fixed parameters for the svm
kernel_svm_no = 2
kernel_svm = kernel_2
deg = 2
gamma = 0

percentages = list(range(0, 100))

total_error = {}

for percentage in percentages:
    total_error[percentage] = []

    for i in range(0, 50):
        
        # Get training and testing
        seed = random.randint(0, 9999999)
        training_x, training_y, test_x, test_y = select_percentage(Xs0, Xs1, percentage, seed)
        
        # Training
        target_list = training_y        
        P_mat = pre_cal(training_x, target_list, kernel_svm_no, deg, gamma)        
        alpha, b, sv_indices = train(training_x, target_list, kernel_svm, slack, deg, gamma)
        
        # Testing
        targets_list = test_y
        test_predict, error = test_score_svm(test_x, targets_list, training_x, training_y, kernel_svm_no, alpha, b, deg, gamma )
        total_error[percentage].append(error / size_test)
        
    print("Mean Test Error for percentage ", percentage, np.mean(total_error[percentage]))
    
    with open("./inv_different_distributions.pickle", "wb") as fp:
        pickle.dump(total_error, fp)
        fp.close()
        
        
       

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean Test Error for percentage  0 0.5102040816326531
Mean Test Error for percentage  1 0.5102040816326531
Mean Test Error for percentage  2 0.5102040816326531
Mean Test Error for percentage  3 0.5096836734693876
Mean Test Error for percentage  4 0.5096836734693876
Mean Test Error for percentage  5 0.5091326530612246
Mean Test Error for percentage  6 0.5091836734693878
Mean Test Error for percentage  7 0.5091632653061225
Mean Test Error for percentage  8 0.508591836734694


# Text dataset

In [ ]:
import pickle
from numpy.core import multiarray
import pandas as pd
from scipy.spatial.distance import minkowski

In [ ]:
with open("../Dataset/textDataset.pickle", "rb") as fp:
    text = pickle.load(fp, encoding="latin-1")

In [ ]:
# SVM compatible outputs
for i in range(0, len(text)):
    if text[i][1] == 0:
        text[i][1] = -1.
    else:
        text[i][1] = 1.


# Separate Targets from features
Xs = []
labels = []
print("textdataset size before filtering ",len(text))
for i in range(0, len(text)):
    #THRESHOLD => Check if the dimensionality of the data is inf to 360 to filter out some words.
    if(len(text[i][0].data) < 360):
        Xs.append(text[i][0].data)
        labels.append(text[i][1])

In [ ]:
print("new textDatasetsize ",len(Xs))

In [ ]:
#extend NDARRAYS length
def extend(list_to_extend, target_len):
    '''
    a = np.ndarray((2,), buffer=np.array([1,2,3])
    >>> array([2, 3])
    extend(a, 5)
    >>> array([2, 3, 0, 0, 0])
    '''
    return np.append(list_to_extend,np.zeros((target_len-len(list_to_extend),)))

def max_len_of_lists(lists):
    maxLen = len(lists[0])
    for i in range(1, len(lists)):
        listLen = len(lists[i])
        if listLen > maxLen:
            maxLen = listLen
    return maxLen

In [ ]:
#extending the arrays
maxLen = max_len_of_lists(Xs)
for i in range(0, len(labels)):
    Xs[i] = extend(Xs[i],maxLen)

In [ ]:
#svm parameters
n_labeled = 987
size_test = 128

In [ ]:
'''
features : filtered data from the text dataset
kernel_k : Which kernel to use for the K_matrix in the paper
                - linear
                - step
                - linear step
                - polynomial
'''
n_clusters = 2 
# Building k_matrix
features = Xs
kernel_k = polynomial
lambdaCut = 5
k_matrix = build_K(lambdaCut, kernel_k, features, n_clusters, sigma = 0.55) 

In [ ]:
'''
divide the the k_matrix in subsets:
    - first => X_k[0] : 987 points used for training.
    - second, third ...etc: partitions of size 'size_test' used for testing.
'''

#first partition: to change size => change the value of n_labeled
X_k = {}
labels_k = {}

X_k[0] = []
labels_k[0] = []
for i in range(0, n_labeled):
    X_k[0].append(k_matrix[i])
    labels_k[0].append(labels[i])
X_k[0] = np.array(X_k[0])
labels_k[0] = np.array(labels_k[0])


#other partitions: to change size => change the value of size_test
partition_size = size_test
begin = n_labeled
end = len(Xs)-1
partitions = 100

for part_index in range(1, partitions):
    X_k[part_index] = []
    labels_k[part_index] = []
    for i in range(0, partition_size):
        #select random data
        data_index = random.randint(begin, end)
        
        X_k[part_index].append(k_matrix[data_index])
        labels_k[part_index].append(labels[data_index])
        
    X_k[part_index] = np.array(X_k[part_index])
    labels_k[part_index] = np.array(labels_k[part_index])

In [ ]:
#info about X_k
print("size of dict X_k: ",len(X_k))
for i in range(0,len(X_k)):
    print("array ",i," is of size: ",len(X_k[i]))


In [ ]:
'''
kernel_svm : Which kernel to use in the svm
                - 1: linear
                - 2: polynomial
                - 3: rbf
slack : slack value in the svm
n_labeled : Number of labeled instances
'''

slack = 10
kernel_svm_no = 2
kernel_svm = kernel_2
degree = 2
gamma = 1.1

#training
total_error = {}
fold = 0
print("fold: ",fold)
total_error[fold] = 0

# pre_cal(features_list, target_list, kernel, deg, gamma)
target_list = labels_k[fold]

P_mat = pre_cal(X_k[fold], target_list, kernel_svm_no, degree, gamma)
print("HERE1")

# train(features_list, target_list, kernel, slack, degree, gamma)
alpha, b, sv_indices = train(X_k[fold], target_list, kernel_svm, slack, degree, gamma)
print("HERE2")

In [ ]:
#test
for test_fold in X_k:
    if test_fold != fold:
        # test_score_svm(test_features, test_targets, train_features, train_targets, kernel, alpha_list, b, deg=0, gamma=0 )
        targets_list = labels_k[test_fold]
        test_predict, error = test_score_svm(X_k[test_fold], targets_list, X_k[fold], labels_k[fold], kernel_svm_no, alpha, b, degree, gamma )
        total_error[test_fold] = error / size_test
        print("fold number ", test_fold, "has an error of ", total_error[test_fold])


print('Mean of errors: ', sum(v for k,v in total_error.items())/len(total_error))

#  This part is the random generated data - for people with trust issues

In [ ]:
def create_clusters(lbA1, ubA1, lbA2, ubA2, lbB, ubB, spreadA, spreadB):
    classA = np.concatenate((np.random.randn(10,2) * spreadA + [lbA1, ubA1],
                           np.random.randn(10, 2) * spreadA + [lbA2, ubA2]))
    classB = np.random.randn(20, 2) * spreadB + [lbB, ubB]
    inputs = np.concatenate((classA, classB))
    targets = np.concatenate(
                (np.ones(classA.shape[0]),
                -np.ones(classB.shape[0])))

    N = inputs.shape[0]

    permute = list(range(N))
    random.shuffle(permute)
    inputs = inputs[permute, :]
    targets = targets[permute]
    return inputs, targets, N, classA, classB

In [ ]:
def generate_data():

    #generating data
    classA=np.concatenate((np.random.randn(10,2)*0.5+[1.5,0.5],np.random.randn(10,2)*0.5+[-1.5,0.5]))
    classB=np.random.randn(10,2)*0.5+[0.0,-0.5]

    inputs=np.concatenate((classA,classB))
    targets=np.concatenate((np.ones(classA.shape[0]),-np.ones(classB.shape[0])))

    N=inputs.shape[0]  #number of rows(samples)

    permute=list(range(N))
    random.shuffle(permute)

    #features_list and target_list are global variables
    inputs=inputs[permute,:]  
    targets=targets[permute]

    return (classA,classB,inputs,targets)

In [ ]:
kernel = 2
degree = 2
gammas = 0.5

cA,cB,features_list,target_list=generate_data()  #this generates the data
N = features_list.shape[0]
x=np.linspace(10,1000,1)
for slack in x:
    
    P_mat = pre_cal(features_list, target_list, kernel, degree, gammas)  #p matrix is initialised here
    print(features_list)
    filtered_alphas, b, sv_indices = train(features_list, target_list, kernel_2, slack, degree, gammas)
    plot(cA, cB, filtered_alphas, b, kernel, slack, degree, gammas)

In [ ]:
#Jade generated data
data_list, target_list, N, cA, cB = create_clusters(random.uniform(-5,5), random.uniform(-5,5), random.uniform(-5,5), 
                                                         random.uniform(-5,5), random.uniform(-5,5), random.uniform(-5,5), random.uniform(0,1), random.uniform(0,1))
x=np.linspace(10,1000,1)
for slack in x:

    P_mat = pre_cal(data_list, target_list, kernel, deg, gamma)  #p matrix is initialised here
    filtered_alphas, b = train(data_list, target_list, kernel, slack)
    plot(cA, cB, filtered_alphas, b, kernel, slack)